In [1]:
import random
import copy as cp
import itertools
import gym
import numpy as np
from gym import spaces
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.feature_extraction import FeatureHasher
import pandas as pd


We will do everything based on indexes in a list array like the following to facilitate accessing it:

0 : Black

1 : Blue

2 : Green

3 : Red

4 : White

5 : Joker 

### Agent:

### ENVIRONMENT from OpenAI

In [2]:


class SplendorEnv(gym.Env):
    MAX_GEMS_SUPPLY=10 #general attribute to call and compare againtst gems
    MAX_JOKERS=3
    def __init__(self, card_supply, deck_nobles, initial_gems =[4,4, 4, 4, 4,3], gem_reserve=None, cards_player=None, players_state=None, nobles=None):
        '''Let's define the initial state and variables.'''
        num_card_tiers = 3
        cards_per_tier = 4
        num_token_colors = 5
        num_nobles = 3
    
        #creates the card decks
        deck=cp.deepcopy(card_supply) #since we are using pop we need to define a new list removing references. 
        self.deck_1 = deck[0]
        self.deck_2 = deck[1]
        self.deck_3 = deck[2]

        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()

        #creates the noble board
        self.nobles = []
        self.CreateNobles(deck_nobles)

        #Create the gems
        self.gem_reserve= initial_gems
        self.gems = [0,0,0,0,0,0]

        #create the gems combinations
        self.binary_lists_with_sum_of_3 = list(filter(lambda binary_list: sum(binary_list) == 3, itertools.product([0, 1], repeat=5)))
        self.binary_lists_with_sum_of_2 = list(filter(lambda binary_list: sum(binary_list) == 2, itertools.product([0, 1], repeat=5)))
        
        
        ##Valid actions:
        self.all_actions = []
        self.create_all_actions()

        self.valid_actions=[0]*len(self.all_actions)

        self.score=0

        ##For our agent:
        self.player_cards = cards_player if cards_player is not None else [] # cars in the player power
        self.player_reserved_cards = [] #cards reserved by the player
        self.nobles = nobles if nobles is not None else [] #nobles the player has
        self.buying_power = self.gems # buying power is initializes equal to the amount of gems of each kind

        ##Binary list for combinations of gems:
        self.binary_lists_with_sum_of_3 = list(filter(lambda binary_list: sum(binary_list) == 3, itertools.product([0, 1], repeat=5)))
        self.binary_lists_with_sum_of_2 = list(filter(lambda binary_list: sum(binary_list) == 2, itertools.product([0, 1], repeat=5)))

        ##Simpler version:
        self.observation_space = gym.spaces.Dict({
            'player_gems': spaces.Box(low=0, high=7, shape=(num_token_colors+1,), dtype=np.integer), #array 6 elements
            'gems_supply': spaces.Box(low=0, high=7, shape=(num_token_colors+1,), dtype=np.integer), #array 6
            'player_cards': spaces.Box(low=0, high=np.inf, shape=(num_token_colors,), dtype=np.integer), #array 5
            'player_score': spaces.Discrete(30), # integer
            'nobles': spaces.Tuple(tuple([spaces.Tuple((spaces.MultiDiscrete([1,1,1,1,1]), spaces.Discrete(5))) for _ in range(num_nobles)])), #cost and points per noble
            'player_reserved':spaces.Discrete(2),
            'valid_actions':spaces.Discrete(57)
            })
        
        ##Max purchase : 12 cards
        ##Max reserve = 12 cards
        ##Max reserved purchase = 3
        ###Note for max_gem combination:
            #3 out of 5 gems of different colors = 10 options
            #2 out of 5 of same color = 5 options
            #2 out of 5 of different color = 10  options
            # 1 out of 5 =5 options
            ##Total = 30  options.
        ##Total: 57 
        # Define action space
        ##For now we are not using this at all... no idea.
        self.action_space = spaces.Dict({
            'action_type': spaces.Discrete(4),  # Action type (0-3)
            'purchase_card': spaces.Tuple((
                spaces.Discrete(num_card_tiers),    # Card tier (0-2)
                spaces.Discrete(cards_per_tier),    # Card index (0-cards_per_tier)
            )),
            'reserve_card': spaces.Tuple((
                spaces.Discrete(num_card_tiers),    # Card tier (0-2)
                spaces.Discrete(cards_per_tier),    # Card index (0-cards_per_tier)
            )),
            'buy_reserved': spaces.Discrete(3),  # Action type (0-3)
            'pick_tokens': spaces.MultiBinary(num_token_colors)  # Tokens to pick
        })

   
    def UpdateCardBoard(self):
        '''Randomly shuffle each deck and take the first 3 cards to place them on the board
        Updates boards to 3 if there are less than 3 cards'''
        random.shuffle(self.deck_1)
        random.shuffle(self.deck_2)
        random.shuffle(self.deck_3)
        #update cards rank 1
        while len(self.board_1) <4:
            if len(self.deck_1) ==0:
                break
            self.board_1.append(self.deck_1.pop(0))
            
        #update cards rank 2
        while len(self.board_2) <4:
            if len(self.deck_2) ==0:
                break
            self.board_2.append(self.deck_2.pop(0))
            
        #update cards rank 3
        while len(self.board_3) <4:
            if len(self.deck_3) ==0:
                break
            self.board_3.append(self.deck_3.pop(0))

    def create_all_actions(self):
        '''
        create all the actions possible
         '''
        #all the buying actions possible

        card_buy = []
        for board in [self.board_1, self.board_2, self.board_3]:   
                for card in board:
                    
                    card_buy.append((card.level-1,board.index(card))) # we append a tuple containing level(0-2),index in board(0-3)
        self.all_actions.extend(  [('buy card', x) for x in card_buy])

        #all the reseravations possible possible
        reserve_cards = []
        for board in [self.board_1, self.board_2, self.board_3]:
                for card in board:
                            reserve_cards.append((card.level-1,board.index(card)))
        self.all_actions.extend( [('reserve', x) for x in reserve_cards])  

        #buy all the cards possible in our reserved cards
        for i in range(self.MAX_JOKERS):
             self.all_actions.append(('buy reserved', i))
        
        #append all the possible token combinations
        self.all_actions.extend([('take', x) for x in self.binary_lists_with_sum_of_2])
        self.all_actions.extend([('take', x) for x in self.binary_lists_with_sum_of_3])
        self.all_actions.extend([('take', x) for x in [[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1], [2,0,0,0,0], [0,2,0,0,0], [0,0,2,0,0], [0,0,0,2,0], [0,0,0,0,2]]])
        #last resort actions
        negative_gems = [[ -x for x in inner_list] for inner_list in self.binary_lists_with_sum_of_3 ]
        self.all_actions.extend([('take',x) for x in negative_gems ])     

    
    
    def CreateNobles(self, deck_nobles):
        '''Shuffle the deck of Nobles and get the first 3 cards - no replacement here.'''
        deck_n=cp.deepcopy(deck_nobles)
        random.shuffle(deck_n)

        #update cards rank 1
        while len(self.nobles) <4:
            self.nobles.append(deck_n.pop(0)) #the issue for reseting is that when we pop we modify the original
    
    def UpdateBuyingPower(self): 
        ''' Buying power is gems + cards bought of each color'''
        self.buying_power =cp.deepcopy(self.gems)
        for card in self.player_cards:
            self.buying_power[card.color] +=1 # if you have bought a card of said color you get an extra buying unit for that color.

    def get_legal_actions(self):
        '''I'll try to get all legal actions as indexes:
        Buy Card , (i,j) #specifies which card to buy
        Reserve Card, (i,j) #card to reserve
        Buy reserved (i) # index in reserved cards.
        Take Gems, [x,x,x,x,x] #gems to take
        ''' 
        #

        ##First we get all legal actions:
        legal_actions=[] #57 available actions:
        ##Check cards that can be bought based on updated buying power'''
        card_buy = self.get_purchasable_cards()
        legal_actions.extend(card_buy)
        ##Now reservable card(should be all as long as we have enough space and gems)'''
        reserve_cards=self.get_reservable_cards()
        legal_actions.extend(reserve_cards)
        ##We get the purchasable reserved cards:
        buyable_reserved_cards=self.get_purchasable_reserved()
        legal_actions.extend(buyable_reserved_cards)
        ##Takeable gems combinations:
        gem_buy=self.get_takable_gems()
        legal_actions.extend(gem_buy)
        #return self.transform_legal_actions(legal_actions) #to transform list of tuples to dictionary of 4 main actions.
        if len(legal_actions)==0:
             print('empty legal actions') #if we have no available actions because we have 10 gems and no cards to buy we reshuflle or remove gems,
             print('gem reserve', self.gem_reserve, 'gems', self.gems)
             last_resort_action=[]
             for gem in self.gems[:5]:
                  if gem>0 and sum(last_resort_action) > -3:#we remove 1 gem of each gem reserve with more than 0 
                       last_resort_action.append(-1)
                  else:
                       last_resort_action.append(0)
             legal_actions=[('take',last_resort_action)]
        
        self.valid_actions=[0]*len(self.all_actions)
        for x in legal_actions:
             self.valid_actions[self.all_actions.index(x)] = 1
        return legal_actions           
    
    
    def get_purchasable_cards(self):
        '''We get all purchasable cards based on buying power(gems + cards) and return the index (level, column) in list format'''
        card_buy=[]
        for board in [self.board_1, self.board_2, self.board_3]:   
                for card in board:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])
                    if over <= self.buying_power[-1]:
                        card_buy.append((card.level-1,board.index(card))) # we append a tuple containing level(0-2),index in board(0-3)
        return  [('buy card', x) for x in card_buy]

    def get_reservable_cards(self):
        '''Check reservable cards based on number of gems and number of reserved cards.'''
        reserve_cards = []
        if sum(self.gems) <10 and len(self.player_reserved_cards) < self.MAX_JOKERS:
            for board in [self.board_1, self.board_2, self.board_3]:
                    for card in board:
                            reserve_cards.append((card.level-1,board.index(card)))
        return [('reserve', x) for x in reserve_cards]         
        
    def get_purchasable_reserved(self):
        '''Checks fi we can buy our reserved cards and return index of reserved card purchasable'''
        buyable_reserved_cards = []
        for card in self.player_reserved_cards:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])

                    if over <= self.buying_power[-1]:
                        buyable_reserved_cards.append(self.player_reserved_cards.index(card)) # returns index in reserved cards 0-2
        return [('buy reserved', x) for x in buyable_reserved_cards]

    def get_takable_gems(self):
        '''We check all possible combinations of gem taking doubles and triples.'''
        #make the list of gems currently takable
        doable_3gems = [] #all possible combinations for 3 gems
        for comb in self.binary_lists_with_sum_of_3:
            if  min([self.gem_reserve[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_3gems.append(comb)

        doable_2gems = [] #possible combinations for 2 gems (same color)
        for comb in self.binary_lists_with_sum_of_2:
            if  min([self.gem_reserve[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_2gems.append(comb)        
        ##max 10 combinations for 2 and 3 so 20 in total.
        gem_buy= []

        # If gem supply is 10, don't consider taking gems
        if sum(self.gem_reserve) ==SplendorEnv.MAX_GEMS_SUPPLY:
            return [] # don't return gems to take.
        
        #append the list of 3 gems if the player has already 7 gems or less
        if sum(self.gem_reserve) <= 7:
            gem_buy.extend(doable_3gems)

        #append the list of 2 gems if the player has already 8

        if sum(self.gem_reserve) == 8:
            gem_buy.extend(doable_2gems)
        
        # append the list of 1 gem if the player has already 9
        
        #TO RECODE TO LIMIT THE SUM TO THE INITIAL MAX NUMBER OF GEMS
        if sum(self.gem_reserve) == 9:
            gem_buy.extend([[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]])

        #Appends the duos
        ##If the supply of gems is greater than 3 we can alwasy take 2 of the same gems.
        if sum(self.gems)<=8:
            for x in range(len(self.gem_reserve[:5])):
                    list_empty = [0,0,0,0,0]
                    if self.gem_reserve[x] >3:
                        list_empty[x] = 2
                        if list_empty != [0,0,0,0,0]:
                            gem_buy.append(list_empty)

        return [('take', x) for x in gem_buy]
    
    #VALIDATE THAT THIS IS USELESS
    '''
    def transform_legal_actions(self):
        actions_dict = {'buy_card': [], 'reserve_card': [], 'buy_reserved': [], 'pick_tokens': []}
        
        for index_list in range(len(self.valid_actions)):
                if self.valid_actions[index_list] ==1:
                    action = self.all_actions[index_list]
                    action_type = action[0]
                    params = action[1]
                    
                    if action_type == 'buy card':
                        actions_dict['buy_card'].append(params)
                    elif action_type == 'reserve':
                        actions_dict['reserve_card'].append(params)
                    elif action_type == 'buy reserved':
                        actions_dict['buy_reserved'].append(params)
                    elif action_type == 'take':
                        actions_dict['pick_tokens'].append(params)
                        
        return actions_dict
        '''
    
    def execute_action(self, action): 
        '''Once an actions has been chosen we execute it by calling the action function'''

        # execute the specified action and update the state
        action_type, action_params = action
        if action_type == 'buy card':
            index = action_params
            self.buy_card(index)

        elif action_type == 'buy reserved':
            index = action_params
            self.buy_reserved_card(index)

        elif action_type == 'reserve':
            index = action_params
            self.reserve_card(index)

        elif action_type == 'take':
            gems = action_params
            self.take_gems(gems)
    

    def buy_card(self, index):
        '''We buy the selected card and update the environment by removing the bought card from the board, updating player's gems and supply gems'''
        # update gem and card supply and player's state when a card is bought
        #we get the card from the index:
        card=None
        while card is None:
            for board in [self.board_1, self.board_2, self.board_3]:
                        for cards in board:
                             if index == (cards.level-1,board.index(cards)):
                                  card=cards
                                
        #Before adding card to players hand we need to check how much buying power do our current cards give us:
        
        #extract the colors of the current cards
        cards_color = [x.color for x in self.player_cards]

        #get our current color portofolio = buying power without gems
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        #add card to players cards
        self.player_cards.append(card)

        ##Now we remove it from the board
        #print('cards power:',cards_sum)
        #print('cost:',card.cost)
        if card.level ==1:
             self.board_1.remove(card)
        
        elif card.level ==2:
             self.board_2.remove(card)

        elif card.level ==1:
             self.board_3.remove(card)

        ##First check from the cost we substract our current free buying power.
        new_cost=[0,0,0,0,0]
        for i in range(len(card.cost)):
            new_cost[i]=max(0,card.cost[i]-cards_sum[i])
        
        #print('Discounted card cost:',new_cost)


        #compute required tokens
        over = 0
     
        '''I'm using gem_reserve instead of buying power, I think it will be better to keep the values from gem reserve and cards separated to know what gems to resupply and which not'''
        for i in range(len(card.cost)):
            #update to know if we have to spend jokers

            over = max(0, new_cost[i]- self.gems[i]) #jokers needed for this color
            gem_transaction = max(0, new_cost[i] - over)# after substracting cards + joker we get the amount to pay.
            if over>self.gems[5]:
                 #print(over,self.gem_reserve)
                 raise ValueError("Too many jokers")
            
            #print('gem_transaction?',gem_transaction)
            self.gems[i] -= gem_transaction
            self.gem_reserve[i]  += gem_transaction
            self.gems[5]-=over
            self.gem_reserve[5]+=over
        #print('New gem reserve:',self.gem_reserve)
        #print('new gem supply:',self.gems)

    def reserve_card(self, index):
        '''We reserve the selected card and remove it from the board'''
        ##Get the card:
        card=None
        while card is None:
            for board in [self.board_1, self.board_2, self.board_3]:
                        for cards in board:
                             if index == (cards.level-1,board.index(cards)):
                                  card=cards
        #add card to players reserved cards
        self.player_reserved_cards.append(card)

        #get rid of the card on the board
        if card.level ==1:
             self.board_1.remove(card)
        
        elif card.level ==2:
             self.board_2.remove(card)

        elif card.level ==3:
             self.board_3.remove(card)

        #get a joker
        self.gems[5] += 1 # add joker to hand.
        self.gem_reserve[5]-=1 # added removal of joker from board

    def buy_reserved_card(self, index):
        '''We buy the selected reserved card and update the environment'''
        ##Get reserved card:
        card=None
        while card is None:
            for cards in self.player_reserved_cards:            
                if index == self.player_reserved_cards.index(cards):        
                     card=cards



        cards_color = [x.color for x in self.player_cards]

        #get our current color portofolio = buying power without gems
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        new_cost=[0,0,0,0,0]
        for i in range(len(card.cost)):
            new_cost[i]=max(0,card.cost[i]-cards_sum[i])
        
        #print('Discounted card cost:',new_cost)
        # update gem and card supply and player's state when a card is bought
        #add card to players cards
        self.player_cards.append(card)

        #compute required tokens
        over = 0
        for i in range(len(card.cost)):
            #update to know if we have to spend jokers
            over=max(0,new_cost[i]-self.gems[i])
            #update the number of reamining gems
            if over>self.gems[5]:
                 raise ValueError("Too many jokers needed")
            gem_transaction = max(0,new_cost[i]-over)
            self.gems[i] -=  gem_transaction
            self.gem_reserve[i]  += gem_transaction
            self.gem_reserve[5]+=over
            self.gems[5]-=over

    def take_gems(self, gems_array):
        '''Add gems from supply to player's reserve and remove it from supply.'''

        # simply add the gems to the players count
        for i in range(len(gems_array)):
            self.gems[i] += gems_array[i]
            self.gem_reserve[i] -= gems_array[i]


    def step(self, action):
  
        self.execute_action(action)
        self.UpdateBuyingPower()
        self.UpdateCardBoard()

        # Calculate reward
        reward,score,done = self.calculate_reward()

        #We compute the new legal actions for the next state to add to the next obs
        self.get_legal_actions()

        ##Update observation:
        next_obs = {
        'player_gems': self.gems,  # player has no gems initially
        'gems_supply': self.gem_reserve,  # 4 gems of each color except for gold, which has 5
        'player_cards': self.player_cards,  # player has no cards initially
        'player_score': self.score,  # player has no score initially
        'nobles':self.nobles, 
        'player_reserved':self.player_reserved_cards,
        'valid_actions':self.valid_actions,
        'all_actions': self.all_actions
        }
        # Return the next state, reward, done, and additional info
        return next_obs, reward, done, {}

    def calculate_reward(self):
        '''Count points and win condition'''
        reward=0
        points =  0
        win = False
        for cards in self.player_cards:
              points +=cards.points
        
        for noble in self.nobles:
             points +=noble[1]
        if points>self.score:
             reward=5
        else:
             reward=-1
        if points >= 5:
             win = True
             reward =100
        
        self.score = points
        return reward,points, win



    def reset(self, new_deck,new_deck_nobles, reset_gems = [4,4,4,4,4,3]):
        '''Reset function, re-initializes all environmental variables:
        Set score to 0, reset supply, decks, boards, reserved cards, nobles etc.'''
        ###Reset all variables and rebuild:
        ##Decks:
        self.deck_1=[]
        self.deck_2=[]
        self.deck_3=[]

        #creates the card decks
        self.deck_1 = cp.deepcopy(new_deck[0])
        self.deck_2 = cp.deepcopy(new_deck[1])
        self.deck_3 = cp.deepcopy(new_deck[2])
        ##Boards:
        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()
        ##Nobles:
        #creates the noble board
        self.nobles = []
        self.CreateNobles(new_deck_nobles)
        ##Gems:
        #Create the gems
        self.gems=[0,0,0,0,0,0]
        ##Agentwise:
        self.score=0
        # Reset the agent's variables
        ##For our agent:
        self.gem_reserve = reset_gems # gems in the agents hand initializes as 0
        self.player_cards = [] # cars in the player power
        self.player_reserved_cards = [] #cards reserved by the player
        self.nobles = [] #nobles the player has
        self.buying_power = self.gems # buying power is initializes equal to the amount of gems of each kind

        
        ##Update legal actions:
        self.get_legal_actions()

        ###OBS:
        obs = {
        'player_gems': self.gems,  # player has no gems initially
        'gems_supply': self.gem_reserve,  # 4 gems of each color except for gold, which has 5
        'player_cards': self.player_cards,  # player has no cards initially
        'player_score': self.score,  # player has no score initially
        'nobles':self.nobles, 
        'player_reserved':self.player_reserved_cards,
        'valid_actions':self.valid_actions,
        'all_actions': self.all_actions
        #'nobles': tuple([(self.nobles[0].requirements, self.nobles[0].points), (self.nobles[1].requirements, self.nobles[1].points),(self.nobles[2].requirements, self.nobles[2].points)])  # example noble cost and points per noble
        }

        return obs

    def render(self, mode='human'):
        pass

    def close(self):
        pass


In [3]:
class Card:
    def __init__(self, level, color, cost, points):
        '''Cards are defined by level color cost and points they give'''
        #int ranges from 1 to 3
        self.level = level
        #We would keep as int based on this mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}(alphabetical order)
        self.color = color
        #We do cost as an array [black, ...., white] [0, 2, 1, 0, 0]
        self.cost = cost
        #int, ranges from 0 to 5
        self.points = points

class Noble:
    def __init__(self, requirements, points):
        '''For nobles we only care about requirements and points you get'''
        #We do requirements as an array [black, ...., white] [3, 3, 3, 0, 0]
        self.requirements = requirements
        #int ranges from 3 to 4
        self.points = points

class GemColor:
    def __init__(self, name):
        self.name = name

## import cards

In [4]:
#mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}

cards_1 = []
cards_2 = []
cards_3 = []

cards = pd.read_csv('card_data.csv').fillna(0)
for index, row in cards.iterrows():
    #get cost
    cost = [row['cost_black'], row['cost_blue'], row['cost_green'], row['cost_red'], row['cost_white']]
    cost =  list(map(int, cost))
    #map to get color
    color_dict = {'black':0, 'blue' :1, 'green':2, 'red': 3, 'white': 4}
    color = color_dict[row['color']]

    card = Card(row['tier'], color, cost, row['points'])

    if row['tier'] ==1:
        cards_1.append(card)

    elif row['tier'] ==2:
        cards_2.append(card)

    else :
        cards_3.append(card)

# create nobles
nobles= [Noble([3,3,0,0,3], 3),
Noble([3,0,3,3,0], 3),
Noble([0,4,0,4,0], 3),
Noble([0,0,4,0,4], 3),
Noble([4,0,0,0,4], 3),
Noble([0,0,0,5,3], 3)]

# create SplendorState
'''Note, card supply and nobles are list of lists of lengths 3 and 1 respectively.'''
##Get the card supply:
card_supply = [cards_1, cards_2, cards_3]

In [5]:
import gym

env = SplendorEnv(card_supply, nobles)
obs = env.reset(card_supply, nobles)
print(obs['all_actions'])
print('obs:',obs)
done = False
actions_list=[]
while not done:
    #random_action = random.choice(obs['valid_actions'])
    list_actions =[]
    for x in range(len(obs['valid_actions'])):
        if obs['valid_actions'][x] ==1:
            list_actions.append(obs['all_actions'][x])
    random_action=random.choice(list_actions)   
    actions_list.append(random_action)
    n_obs,reward,done,_= env.step(random_action)
    print(random_action)
    print('Next_obs:',n_obs)
    obs=n_obs
    env.render()
print(f'Fernando made {env.score} points in {len(actions_list)} turns, by making the following actions {actions_list}' )
env.close()


[('buy card', (0, 0)), ('buy card', (0, 1)), ('buy card', (0, 2)), ('buy card', (0, 3)), ('buy card', (1, 0)), ('buy card', (1, 1)), ('buy card', (1, 2)), ('buy card', (1, 3)), ('buy card', (2, 0)), ('buy card', (2, 1)), ('buy card', (2, 2)), ('buy card', (2, 3)), ('reserve', (0, 0)), ('reserve', (0, 1)), ('reserve', (0, 2)), ('reserve', (0, 3)), ('reserve', (1, 0)), ('reserve', (1, 1)), ('reserve', (1, 2)), ('reserve', (1, 3)), ('reserve', (2, 0)), ('reserve', (2, 1)), ('reserve', (2, 2)), ('reserve', (2, 3)), ('buy reserved', 0), ('buy reserved', 1), ('buy reserved', 2), ('take', (0, 0, 0, 1, 1)), ('take', (0, 0, 1, 0, 1)), ('take', (0, 0, 1, 1, 0)), ('take', (0, 1, 0, 0, 1)), ('take', (0, 1, 0, 1, 0)), ('take', (0, 1, 1, 0, 0)), ('take', (1, 0, 0, 0, 1)), ('take', (1, 0, 0, 1, 0)), ('take', (1, 0, 1, 0, 0)), ('take', (1, 1, 0, 0, 0)), ('take', (0, 0, 1, 1, 1)), ('take', (0, 1, 0, 1, 1)), ('take', (0, 1, 1, 0, 1)), ('take', (0, 1, 1, 1, 0)), ('take', (1, 0, 0, 1, 1)), ('take', (1, 0,

C:\Users\oliha\AppData\Roaming\Python\Python39\site-packages\gym\spaces\box.py:84: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  self.dtype = np.dtype(dtype)


### DQN:

In [6]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, valid_actions, all_actions):
        #if np.random.rand() <= self.epsilon:
        #    return random.choice(valid_actions)
        act_values = self.model.predict(state)[0]
        index_list = 0
        max_value = -np.inf
        for x in range(len(act_values)):
            if (act_values[x] >max_value) and (valid_actions[x]==1):
                index_list  = x
                max_value = act_values[x]

       
        print('act_values = ', act_values)
        print('all-actions (with len) = ', len(all_actions), all_actions)
        print('valid_actions(with len) = ', len(valid_actions), valid_actions)
        print('best actions', all_actions[index_list], 'action value', max_value )

        #while best_action not in valid_actions:
        #    act_values[0][best_action] = -np.inf
        #    best_action = np.argmax(act_values[0])
        return all_actions[index_list]

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        print('minibatch:',minibatch)
        for state, action, reward, next_state, done in minibatch:
            print('state:',state)

            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [7]:
def flatten_state(state):
    flat_state = {}
    
    for idx, value in enumerate(state['player_gems']):
        flat_state[f'player_gems_{idx}'] = value

    for idx, value in enumerate(state['gems_supply']):
        flat_state[f'gems_supply_{idx}'] = value

    for idx, rcard in enumerate(state['player_reserved']):
        flat_state[f'player_reserved_{idx}_level'] = rcard.level
        flat_state[f'player_reserved_{idx}_color'] = rcard.color
        flat_state[f'player_reserved_{idx}_points'] = rcard.points
        for jdx, cost in enumerate(rcard.cost):
            flat_state[f'player_reserved_{idx}_cost_{jdx}'] = cost
        
    flat_state['player_score'] = state['player_score']

    for idx, noble in enumerate(state['nobles']):
        for jdx, req in enumerate(noble.requirements):
            flat_state[f'noble_{idx}_req_{jdx}'] = req
        flat_state[f'noble_{idx}_points'] = noble.points

    for idx, card in enumerate(state['player_cards']):
        flat_state[f'player_card_{idx}_level'] = card.level
        flat_state[f'player_card_{idx}_color'] = card.color
        flat_state[f'player_card_{idx}_points'] = card.points
        for jdx, cost in enumerate(card.cost):
            flat_state[f'player_card_{idx}_cost_{jdx}'] = cost

    return flat_state


In [10]:
from gym import wrappers

EPISODES = 2
hasher = FeatureHasher(n_features=100, input_type='dict')

if __name__ == "__main__":
    state_size = 100  # This is the size of the input vector (preprocessed state) putting it big enough should be fine? 
    action_size = 57  # This is the number of possible actions we need to define it once so the model can work 
    ##It also is the length of action_values in the DQN.
    ###What we need to rework is how to have a list of all actions possibles and then put either 1 or 0 depending on whether they are available.
    agent = DQNAgent(state_size, action_size)
    batch_size = 4

    for e in range(EPISODES):
        state = env.reset(card_supply, nobles)
        flat_state = flatten_state(state)
        state_hashed = hasher.transform([flat_state]).toarray()

        #state = preprocess_state(state)  # Implement a function to preprocess the state
        done = False
        while not done:
            #print(state['valid_actions'])
            action = agent.act(state_hashed, state['valid_actions'], state['all_actions'])
            next_state, reward, done, _ = env.step(action)
            #print(next_state)
            flat_next_state = flatten_state(next_state)
            #print('fns:',flat_next_state)
            next_state_hashed = hasher.transform([flat_next_state]).toarray()

            #next_state = preprocess_state(next_state)
            agent.remember(state_hashed, action, reward, next_state_hashed, done)
            state = next_state
            state_hashed = next_state_hashed
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, env.score, agent.epsilon))

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

        # Optionally, save the agent weights after every 10 episodes
        if e % 10 == 0:
            agent.save("dqn_weights.h5")


1/1 [==============================] - 0s 327ms/step
act_values =  [ 7.4924237e-01  3.6808097e-01  7.1086213e-02 -1.2466626e+00
 -1.7419380e-01 -3.0441433e-02  3.1447834e-01  7.6235825e-01
  1.1223166e-01  2.6697540e-01 -7.1979776e-02 -3.9204977e-02
  3.1170821e-01 -3.8771726e-02 -4.9776384e-01  8.0797786e-01
  6.9209564e-01 -4.5161581e-01 -3.5917678e-01 -4.3003070e-01
 -2.1143958e-02  6.5727383e-01 -3.1428617e-01 -9.4973050e-02
 -3.5838979e-01  8.7836403e-01 -5.9533894e-01 -4.3876928e-01
  6.8140246e-02  4.0725976e-02 -3.5415649e-01  5.8832198e-02
  1.0896073e-01  1.3778134e-01  6.1078250e-01 -1.0554974e-01
 -5.2785307e-01  6.2241423e-01 -3.0015293e-01  3.9207539e-01
 -1.0674901e-01 -1.6720612e-01 -1.6035277e-01  7.6503730e-01
 -8.7016201e-01  1.6242184e-02  2.0140454e-01 -3.2588041e-01
 -1.1437640e-02  1.4211527e-01  1.0525212e-03 -1.6642630e-01
  1.2526330e-02 -7.9012978e-01 -5.0917649e-01  1.1147342e-01
 -6.1340883e-02]
all-actions (with len) =  67 [('buy card', (0, 0)), ('buy card

ValueError: ('take', [0, 0, 0, 0, 0]) is not in list

In [ ]:
import numpy as np
import random

class QLearningAgent:
    def __init__(self, env, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay_rate=0.01):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay_rate = exploration_decay_rate
        self.q_table = {}

    def choose_action(self, state):
        valid_actions = state['valid_actions']
        if np.random.rand() < self.exploration_rate:
            return random.choice(valid_actions)
        else:
            q_values = [self.q_table.get((state['state_hash'], action['action_hash']), 0) for action in valid_actions]
            return valid_actions[np.argmax(q_values)]

    def learn(self, state, action, reward, next_state):
        current_q = self.q_table.get((state['state_hash'], action['action_hash']), 0)
        next_q_values = [self.q_table.get((next_state['state_hash'], next_action['action_hash']), 0) for next_action in next_state['valid_actions']]
        target_q = reward + self.discount_factor * np.max(next_q_values)
        self.q_table[(state['state_hash'], action['action_hash'])] = current_q + self.learning_rate * (target_q - current_q)
        self.exploration_rate *= (1 - self.exploration_decay_rate)

# Initialize the Splendor environment and the Q-learning agent
env = SplendorEnv(card_supply, nobles)
agent = QLearningAgent(env)

# Train the agent for a number of episodes
num_episodes = 5000
for episode in range(num_episodes):
    obs = env.reset(card_supply,nobles)
    done = False
    while not done:
        action = agent.choose_action(obs)
        next_obs, reward, done, _ = env.step(action)
        agent.learn(obs, action, reward, next_obs)
        obs = next_obs

    # Decay exploration rate
    agent.exploration_rate *= (1 - agent.exploration_decay_rate)

# Test the trained agent
obs = env.reset()
done = False
actions_list = []
while not done:
    action = agent.choose_action(obs)
    actions_list.append(action)
    next_obs, reward, done, _ = env.step(action)
    obs = next_obs
    env.render()

print(f'The trained agent made {env.score} points in {len(actions_list)} turns, by making the following actions {actions_list}')
env.close()
